In [1]:
import sys    # 시스템
import os     # 시스템
import pandas as pd    # 판다스 : 데이터분석 라이브러리
import numpy as np     # 넘파이 : 숫자, 행렬 데이터 라이브러리
from bs4 import BeautifulSoup     # html 데이터 전처리
import requests                   # http 요청 라이브러리
import time                       # 시간 지연
from tqdm import tqdm_notebook    # 진행상황 표시

In [11]:
# CSV 파일 불러오기

keyword = pd.read_csv('C:/Users/socsoft/Downloads/internship_kwj/socsoft/법제처키워드/(v)규제_심사_키워드(1156).csv', encoding='utf-8')
keyword

,제목
0,유료방송 승인유효 5년→7년..소유겸영 규제도 대폭 완화
1,"""유료방송 허가 7년・케이블TV 커머스방송 허용"" 개정안 입법예고"
2,당정 “영세가맹점 카드수수료 0.5%로 인하”
3,연매출 30억원까지 신용카드 우대 수수료율 적용 확대
4,"韓-싱가포르, DPA 타결 K콘텐츠 수출 쉬워진다"
...,...
1151,"닻 올린 규제프리존 중기부, 규제자유특구 시행령 입법예고"
1152,​규제자유특구 하위법령 전부개정령안 입법예고
1153,'규제자유특구' 지정절차 특례 규정 개정안 입법예고
1154,"중기부, 책임보험 가입 등 '규제자유특구' 시행령 입법예고"


In [12]:
# 데이터 프레임 열에 이름 없을 경우 이름달아 주기

keyword=keyword.rename(columns={'Unnamed: 0':'순번','0':'제목'})

In [13]:
# 데이터 프레임에서 key 값으로 쓸 열을 리스트 형태로 가져오기
 
key = keyword["제목"]

key

0            유료방송 승인유효 5년→7년..소유겸영 규제도 대폭 완화
1       "유료방송 허가 7년・케이블TV 커머스방송 허용" 개정안 입법예고
2                  당정 “영세가맹점 카드수수료 0.5%로 인하”
3              연매출 30억원까지 신용카드 우대 수수료율 적용 확대
4                韓-싱가포르, DPA 타결 K콘텐츠 수출 쉬워진다
                        ...                 
1151         닻 올린 규제프리존 중기부, 규제자유특구 시행령 입법예고
1152                ​규제자유특구 하위법령 전부개정령안 입법예고
1153            '규제자유특구' 지정절차 특례 규정 개정안 입법예고
1154        중기부, 책임보험 가입 등 '규제자유특구' 시행령 입법예고
1155     핀테크 '규제 샌드박스' 1월부터 사전신청 '4월中 지정 예정'
Name: 제목, Length: 1156, dtype: object

In [14]:
# 결과 저장 코드

news_data = {}    # 전체 크롤링 데이터를 담을 그릇

inlink_url = []   # 네이버 인링크 추출

# 사이트 주소

print('네이버 뉴스 인링크 수집중...')

for i in tqdm_notebook(range(0, len(key))):
    try:
        url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'.format(key[i])
        headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
        resp = requests.get(url, headers=headers)
        soup = BeautifulSoup(resp.text)

        # 뉴스리스트 전체 정보
        n_info = soup.find('ul', class_='list_news')

        # [네이버뉴스] 버튼 있는 첫 뉴스만 저장
        for i in n_info.find_all('a'):
            if 'https://n.news.naver.com/' in i['href']:
                #print(i['href'])
                inlink_url.append(i['href'])
                break
    except:
        inlink_url.append('키워드: {}'.format(key[i]))        
print(len(inlink_url), '개 완료')
print('\n')

# 가져온 네이버 인링크 URL로 네이버뉴스 검색

print('인링크 뉴스데이터 수집중...')
for i in tqdm_notebook(range(0, len(inlink_url))):
    try:
        url = inlink_url[i]
        headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
        resp = requests.get(url, headers=headers)       
        soup = BeautifulSoup(resp.text)
        
        target_info = {}  # 개별 뉴스 내용을 담을 딕셔너리 생성
        
        # 네이버 인링크 뉴스 정보
        news_info = soup.find('div', class_='newsct')

        # 제목 크롤링
        title = news_info.find('h2', 'media_end_head_headline').text

        # 글쓴이 크롤링
        try:
            nickname = news_info.find('em', class_='media_end_head_journalist_name').text
        except:
            nickname = 'None'
        # 날짜 크롤링
        datetime = news_info.find('span', class_='media_end_head_info_datestamp_time _ARTICLE_DATE_TIME').text

        # 내용 크롤링
        contents = news_info.find('div', class_='newsct_article _article_body')
        content1 = contents.find(class_='go_trans _article_content')
        content = content1.text.replace('\n','')
        
        # 언론사 이름 크롤링
        company = news_info.find('img')['title']

        # 크롤링한 글은 target_info라는 딕셔너리에 담음
        target_info['제목'] = title
        target_info['기자명'] = nickname
        target_info['작성일'] = datetime
        target_info['본문'] = content
        target_info['언론사'] = company
        target_info['url'] = inlink_url[i]
        
        # 각각의 글은 dict라는 딕셔너리에 담음
        news_data[i] = target_info
        
    # 에러날 경우 예외 처리
    except:
        #!time.sleep(0.5)
        continue
        
    if len(news_data) % 10 == 0:
        print(i, title)
        
print('수집한 글 갯수: ', len(news_data))

# 크롤링 결과 데이터 프레임으로 만들기
result_df = pd.DataFrame.from_dict(news_data, 'index')

# 데이터 프레임을 엑셀로 저장
result_df.to_excel("naver_news_content.xlsx", encoding='utf-8-sig')

네이버 뉴스 인링크 수집중...


C:\Users\socsoft\AppData\Local\Temp\ipykernel_14544\1444450456.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(0, len(key))):


  0%|          | 0/1156 [00:00<?, ?it/s]

1094 개 완료


인링크 뉴스데이터 수집중...


C:\Users\socsoft\AppData\Local\Temp\ipykernel_14544\1444450456.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(0, len(inlink_url))):


  0%|          | 0/1094 [00:00<?, ?it/s]

9 'OTT 자체등급분류제' 관계부처 이견에 7개월째 공회전
19 인앱결제 강제 매출액 2% 과징금… 구글·애플 `꼼수` 제동
30 [현장기자] 온플법 주도권 여당에 넘긴 부끄러운 공정위·방통위
40 ‘반값 복비’ 전국 시행 첫날, “계약날짜 변경해도 되나요” 눈치싸움
50 19일부터 복비 내린다… 9억 매매 810만원→450만원
60 부동산 중개 수수료 인하 19일부터 시행 확정
70 9억집 사고팔때 810만→450만원…부동산 중개수수료 인하 19일부터 시행[종합]
80 "장판까지 들춰봐야 하나"…손해배상 책임 커지는 중개업계
90 [이코노미 플러스]부동산 중개보수 인하 이르면 이달 시행
100 부동산 중개수수료 이달부터 인하 전망…9억주택 810만→450만원   
110 기존 아파트에도 3년 내 주차장 2%에 전기차 충전기 생긴다
120 "계약 미룰까요"..'11월부터 복비 반값'에 주택거래도 미뤄지나
130 윤종인 "개보법 과징금 기준 완화, 강제조사권은 유지"
141 캠핑카도 렌트 가능해진다…"캠핑문화 활성화 기대"
153 ‘천안 한양수자인 에코시티’ 오늘 무순위 청약
163 어선 감척 지원받고 3년 내 재진입하면 폐업지원금 환수한다
173 지상파까지 중간광고 허용 눈앞…시청권 보호 대책은?
186 지상파에서도 중간광고 본다…심야 주류 PPL은 금지
197 '플랫폼 규제' 사활 건 공정위, 이중규제 우려에 떠는 기업들
207 “용도변경 안 된다?” 갈수록 커지는 ‘레지던스 주거 불가’ 논란 [부동산360]
219 [특별기고] 휴수동행의 마음으로
229 제동장치 없는 '퍼주기 입법' 폭주···의원입법도 '영향평가' 도입해야
239 무순위 '줍줍' 청약 없어진다…발코니 확장 '끼워팔기' 금지
249 수십만 몰리는 '무순위 줍줍청약' 유주택자·외지인은 못한다
259 생활형 숙박시설 첫 규제 …“주거 시설 금지”
269 "소송 한 건에 中企 사망"…'기업 징벌 3법'의 늪
281 30분마다 TV 광고 본다…방통위, 지상파 중간광고 전면 허용
292 [단독]"플